# The Relationship between store Sales, Waste, and Availability for finanacial year 17/18

## Problem Statement

Do stores with high waste and low sales have lower availability

## Overview

Clustering exercise to group stores across the estate by key metric indicators. 

The initial plan for this project is as follows:

1.Source the data 
2.Explore the data
3.Clean the data
4.Prepare the data for analysis
5.Run K-means clustering on the data
6.Evaluate the data

This is my first independent data science project, I am hoping to identify stores where my team will have an opportunity to improve availability for customers

## Sourcing the Data

This data was pulled out of SAP Business Objects using a manually generated query. This query contains no personally identifying information but does contain commercially sensitive data.

## Exploring the Data


In [4]:
# Importing necessary data file and packages for data exploration and analysis

# Import the pandas library in order to read, transform and manipulate the data
import pandas as pd

# Import matplotlib to visualise the data
import matplotlib.pyplot as plt

# Import the sklearn library in order to implement machine learning algorithms. Note, that we only import the 
# functions we'll need from sklearn (cluster & preprocessing).
from sklearn.cluster import KMeans

# From sklearn import a function that allows us to scale the our data
from sklearn.preprocessing import scale

#Importing Numpy to help to clean Data
import numpy as np

# Command that enables us to display the graphs in our Jupyter notebook (right here!)
%matplotlib inline

In [5]:
# Importing the data to analyse
storeinfo = pd.read_csv('waste_sales_avail.csv')

In [6]:
#Looking at the raw data
storeinfo.head()

,Site Name,Site Code (Corp),Area Name,Area Code,STG Name,STG Code,Gross Retail Sales (GBP),Gross Retail Sales (Units),Total Waste (GBP),Total Waste (Units),Availability
0,ABBEY CENTRE BELFAST,6952,NORTHERN IRELAND REGION,ZARE0025,LGE OUT OF TOWN,LGOT,"20,020,168.32","7,979,544.49","1,515,686.76","550,574.23",95.49
1,ABERDEEN,2587,SCOTLAND NORTH & EAST,ZARE0073,CITY CENTRES,CC,"15,054,656.92","6,408,321.82","1,414,956.05","548,783.29",94.64
2,ABERYSTWYTH,1930,SOUTH WALES & WEST,ZARE0080,SMALL TOWN,SMLTO,"5,593,407.67","2,225,648.61","730,088.72","296,854.64",94.2
3,ACORN PARK,1988,HOME COUNTIES SOUTH,ZARE0064,SML OUT OF TOWN,SOOT,"7,133,873.75","2,900,626.34","785,749.61","314,887.46",94.32
4,AINTREE,132,"MERSEYSIDE, NORTH WALES & IOM",ZARE0067,SML OUT OF TOWN,SOOT,"12,945,741.07","5,133,369.09","907,010.92","338,136.53",95.83


In [7]:
#Exploring the type of data in the data file
storeinfo.dtypes

Site Name                     object
Site Code (Corp)               int64
Area Name                     object
Area Code                     object
STG Name                      object
STG Code                      object
Gross Retail Sales (GBP)      object
Gross Retail Sales (Units)    object
Total Waste (GBP)             object
Total Waste (Units)           object
Availability                  object
dtype: object

### Data Types

There are elements of the data that needs to be cleaned. Gross retail sales, Gross Retail Units, Total Waste (GBP), Total Waste (Units) and Availability are all stored as objects. 

The comma's in the sales and waste columns also need to be removed.

In [8]:
# Removing the comma's and checking this has worked
storeinfo['Gross Retail Sales (GBP)'] = storeinfo['Gross Retail Sales (GBP)'].replace(',','', regex = True)
storeinfo['Gross Retail Sales (Units)'] = storeinfo['Gross Retail Sales (Units)'].replace(',','', regex = True)
storeinfo['Total Waste (GBP)'] = storeinfo['Total Waste (GBP)'].replace(',','', regex = True)
storeinfo['Total Waste (Units)'] = storeinfo['Total Waste (Units)'].replace(',','', regex = True)
storeinfo['Total Waste (Units)'].head()

0    550574.23
1    548783.29
2    296854.64
3    314887.46
4    338136.53
Name: Total Waste (Units), dtype: object

In [9]:
storeinfo.dtypes


Site Name                     object
Site Code (Corp)               int64
Area Name                     object
Area Code                     object
STG Name                      object
STG Code                      object
Gross Retail Sales (GBP)      object
Gross Retail Sales (Units)    object
Total Waste (GBP)             object
Total Waste (Units)           object
Availability                  object
dtype: object

In [10]:
#I am unsure why the availability column is classed as an object so I am going to look at the unique values within it
storeinfo.Availability.unique()

array(['95.49', '94.64', '94.2', '94.32', '95.83', '92.54', '94.54',
       '94.95', '94.82', '90.54', '94.51', '93.79', '94.52', '94.8',
       '94.91', '94.61', '93.81', '94.42', '100', '94.53', '95.48',
       '99.26', '90.02', '94.07', '93.49', '95.05', '94.41', '95.73',
       '94.73', '95.41', '95.09', '94.85', '94.29', '95.19', '95.16',
       '95.54', '#DIV/0', '92.89', '94.79', '95.04', '93.13', '93.5',
       '93.16', '94.86', '94.34', '94.37', '94.4', '95.07', '95.11',
       '95.31', '94.58', '95.12', '95.03', '92.43', '94.6', '94.78',
       '95.42', '95.32', '92.07', '95.1', '93.39', '96.31', '92.35',
       '93.17', '94', '95.06', '95.08', '94.72', '95.21', '91.69',
       '99.96', '93.09', '92.34', '93.85', '94.13', '94.77', '95.81',
       '99.74', '94.99', '94.65', '94.23', '93.72', '91.77', '94.43',
       '95.39', '93.65', '92.94', '95.17', '94.44', '94.63', '95.88',
       '92.96', '94.39', '94.08', '95.25', '94.89', '94.74', '91.56',
       '95.14', '94.18', '93.6

In [11]:
# I can see there are non numerical data types within the column which need to be removed - nan and #DIV/0
storeinfo['Availability'] = storeinfo['Availability'].replace('#DIV/0','nan', regex = True)


In [12]:
storeinfo.Availability.unique()

array(['95.49', '94.64', '94.2', '94.32', '95.83', '92.54', '94.54',
       '94.95', '94.82', '90.54', '94.51', '93.79', '94.52', '94.8',
       '94.91', '94.61', '93.81', '94.42', '100', '94.53', '95.48',
       '99.26', '90.02', '94.07', '93.49', '95.05', '94.41', '95.73',
       '94.73', '95.41', '95.09', '94.85', '94.29', '95.19', '95.16',
       '95.54', 'nan', '92.89', '94.79', '95.04', '93.13', '93.5',
       '93.16', '94.86', '94.34', '94.37', '94.4', '95.07', '95.11',
       '95.31', '94.58', '95.12', '95.03', '92.43', '94.6', '94.78',
       '95.42', '95.32', '92.07', '95.1', '93.39', '96.31', '92.35',
       '93.17', '94', '95.06', '95.08', '94.72', '95.21', '91.69',
       '99.96', '93.09', '92.34', '93.85', '94.13', '94.77', '95.81',
       '99.74', '94.99', '94.65', '94.23', '93.72', '91.77', '94.43',
       '95.39', '93.65', '92.94', '95.17', '94.44', '94.63', '95.88',
       '92.96', '94.39', '94.08', '95.25', '94.89', '94.74', '91.56',
       '95.14', '94.18', '93.63',

In [13]:
# Now I need to remove the nan
storeinfo = storeinfo.dropna()

In [14]:
storeinfo.Availability.unique()

array(['95.49', '94.64', '94.2', '94.32', '95.83', '92.54', '94.54',
       '94.95', '94.82', '90.54', '94.51', '93.79', '94.52', '94.8',
       '94.91', '94.61', '93.81', '94.42', '100', '94.53', '95.48',
       '90.02', '94.07', '93.49', '95.05', '94.41', '95.73', '94.73',
       '95.41', '95.09', '94.85', '94.29', '95.19', '95.16', '95.54',
       '92.89', '94.79', '95.04', '93.13', '93.5', '93.16', '94.86',
       '94.34', '94.37', '94.4', '95.07', '95.11', '95.31', '94.58',
       '95.12', '95.03', '92.43', '94.6', '94.78', '95.42', '95.32',
       '92.07', '95.1', '93.39', '96.31', '92.35', '93.17', '94', '95.06',
       '95.08', '94.72', '95.21', '91.69', '93.09', '92.34', '93.85',
       '94.13', '94.77', '95.81', '94.99', '94.65', '94.23', '93.72',
       '91.77', '94.43', '95.39', '93.65', '92.94', '95.17', '94.44',
       '94.63', '95.88', '92.96', '94.39', '94.08', '95.25', '94.89',
       '94.74', '91.56', '95.14', '94.18', '93.63', '94.02', '94.96',
       '93.71', '94.93

In [15]:
#I can now change the datatype from an object to a float
storeinfo['Availability'] = storeinfo['Availability'].astype(float)
storeinfo['Availability'].dtype

dtype('float64')

In [16]:
#Knowing the above code has worked to change the data type in the availability column is will now do the same for the other measure
storeinfo['Gross Retail Sales (Units)'] = storeinfo['Gross Retail Sales (Units)'].astype(float)
storeinfo['Gross Retail Sales (GBP)'] = storeinfo['Gross Retail Sales (GBP)'].astype(float)
storeinfo['Total Waste (GBP)'] = storeinfo['Total Waste (GBP)'].astype(float)
storeinfo['Total Waste (Units)'] = storeinfo['Total Waste (Units)'].astype(float)

In [17]:
#checking the data types
storeinfo.dtypes

Site Name                      object
Site Code (Corp)                int64
Area Name                      object
Area Code                      object
STG Name                       object
STG Code                       object
Gross Retail Sales (GBP)      float64
Gross Retail Sales (Units)    float64
Total Waste (GBP)             float64
Total Waste (Units)           float64
Availability                  float64
dtype: object

In [18]:
storeinfo.head()

,Site Name,Site Code (Corp),Area Name,Area Code,STG Name,STG Code,Gross Retail Sales (GBP),Gross Retail Sales (Units),Total Waste (GBP),Total Waste (Units),Availability
0,ABBEY CENTRE BELFAST,6952,NORTHERN IRELAND REGION,ZARE0025,LGE OUT OF TOWN,LGOT,20020168.32,7979544.49,1515686.76,550574.23,95.49
1,ABERDEEN,2587,SCOTLAND NORTH & EAST,ZARE0073,CITY CENTRES,CC,15054656.92,6408321.82,1414956.05,548783.29,94.64
2,ABERYSTWYTH,1930,SOUTH WALES & WEST,ZARE0080,SMALL TOWN,SMLTO,5593407.67,2225648.61,730088.72,296854.64,94.20
3,ACORN PARK,1988,HOME COUNTIES SOUTH,ZARE0064,SML OUT OF TOWN,SOOT,7133873.75,2900626.34,785749.61,314887.46,94.32
4,AINTREE,132,"MERSEYSIDE, NORTH WALES & IOM",ZARE0067,SML OUT OF TOWN,SOOT,12945741.07,5133369.09,907010.92,338136.53,95.83
